## This notebook does finite difference approaches for a few simple differential equations. 

In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from animate import *
plt.style.use('dynamics_site')
import sys
sys.path.append('../../src/')
from helper_functions import *

## Stiff example 

In [2]:
#define findiff approx:
def getsol(tmax,f0,dt):
    y=[f0]
    x=[0]
    time=0
    while time < tmax:
        f_next = (1-15*dt)*f0
        y.append(f0)
        x.append(time)
        f0 = f_next
        time= time+dt
    return x, y


def generator(dt):
    matplotlib.use('agg')
    # Set up data
    x = np.linspace(0, 2, 100)
    y = np.e**(-15*x)
    newx,newy = getsol(2,1,dt)
    
    fig,ax = plt.subplots(figsize=(7,7))
    plt.plot(x,y,c='k',label='$e^{t}$',zorder=0)
    plt.scatter(newx,newy,c='r',label='numerical f(t)',zorder=1)
    plt.ylabel("f(t)")
    plt.xlabel('t')
    plt.title('$\Delta t =$'+str(np.round(dt,3)))
    plt.legend(loc='lower right')
    plt.ylim(-3,3)
    plt.close() #stops figure from being shown
    return fig

In [3]:
fig=generator(0.01)
fig

<Figure size 700x700 with 1 Axes>

In [4]:
parameter_grid = np.linspace(0.25,0.001,150)
savefigures(generator,parameter_grid,outdir='./example_stiff/')
render('./example_stiff/','example_stiff','gif',cleanup_type='rm')

saving figures


100%|█████████████████████████████████████████| 150/150 [00:25<00:00,  5.95it/s]
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora 

Framerate is None fps
Total Runtime is None s


[Parsed_palettegen_0 @ 0x55bd9014b840] 255(+1) colors generated out of 1250 colors; ratio=0.204000
[Parsed_palettegen_0 @ 0x55bd9014b840] Dupped color: FF1F1F1F
Output #0, image2, to '../temp_palette_example_stiff.png':
  Metadata:
    encoder         : Lavf58.76.100
  Stream #0:0: Video: png, rgba(pc, gbr/unknown/unknown, progressive), 16x16 [SAR 1:1 DAR 1:1], q=2-31, 200 kb/s, 25 fps, 25 tbn
    Metadata:
      encoder         : Lavc58.134.100 png
frame=    1 fps=0.7 q=-0.0 Lsize=N/A time=00:00:00.04 bitrate=N/A speed=0.0265x    
video:1kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: unknown
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa 

## Some still solutions

In [5]:
# Set up data
x = np.linspace(0, 2, 100)
y = np.e**(-15*x)

fig,ax = plt.subplots(figsize=(7,7))
plt.plot(x,y,c='k',label='$e^{t}$',zorder=0)
newx,newy = getsol(2,1,0.25)
plt.plot(newx,newy,c='r',label=r'$\Delta t = 1/5$',zorder=1,ls='--',lw=3)
newx,newy = getsol(2,1,0.125)
plt.plot(newx,newy,c='g',label=r'$\Delta t = 1/10$',zorder=1,ls='--',lw=3)
newx,newy = getsol(2,1,0.0625)
plt.plot(newx,newy,c='b',label=r'$\Delta t = 1/20$',zorder=1,ls='--',lw=3)
plt.ylabel("f(t)")
plt.xlabel('t')
plt.legend(loc='lower right')
plt.ylim(-3,3)
plt.savefig('single_examples.png')

## Nbody Comet Examples

In [6]:
def integrate_rk2_midpoint(system,dt,steps):
    G, mvec, rvec, vvec = load_initials(system)
    Rs = [rvec]
    Vs = [vvec]
    for i in range(steps-1):
        k1_vi = get_acclerations(rvec,mvec)
        k1_ri = vvec

        k2_vi = get_acclerations(rvec+(k1_ri*dt/2),mvec)
        k2_ri = vvec+(k1_vi*dt/2)


        new_vvec = vvec + dt*(1.0*k2_vi)
        new_rvec = rvec + dt*(1.0*k2_ri)

        Rs.append(new_rvec)
        Vs.append(new_vvec)
        rvec = new_rvec
        vvec = new_vvec

    Rs = np.array(Rs)
    Vs = np.array(Vs)
    PE = get_PE(Rs,mvec)
    KE = get_KE(Vs,mvec)
    
    Rs = np.array([Rs[j].T for j in range(len(Rs))])
    Vs = np.array([Vs[j].T for j in range(len(Rs))])
    return Rs,Vs,PE+KE


In [7]:
#preform integration 1
dt1 = 0.01
steps1 = 20000
Rs1,Vs1,Es1 = integrate_rk2_midpoint('../../src/Comet-Sun.npy',dt1,steps1)
xs1,ys1 = Rs1[:,:2,0].T
xp1,yp1 = Rs1[:,:2,1].T
times1 = np.linspace(0,steps1*dt1,steps1)

In [8]:
#preform integration 2
dt2 = 0.05
steps2 = 4000
Rs2,Vs2,Es2 = integrate_rk2_midpoint('../../src/Comet-Sun.npy',dt2,steps2)
xs2,ys2 = Rs2[:,:2,0].T
xp2,yp2 = Rs2[:,:2,1].T
times2 = np.linspace(0,steps2*dt2,steps2)

In [9]:
def makeplot(i):
    timegrid = np.linspace(0,np.max(times2),1000)
    ind1 = np.where(times1 < timegrid[i])
    ind2 = np.where(times2 < timegrid[i])

    fig,ax = plt.subplots(ncols=3,figsize=(30,10))
    ax[0].scatter(xs1[-1],ys1[-1],c='k',marker='*',s=100)
    ax[0].plot(xp1[ind1],yp1[ind1],c='r',alpha=0.3)
    ax[0].scatter(xp1[np.max(ind1)],yp1[np.max(ind1)],c='k',s=100)
    ax[0].set_xlabel("X [AU]")
    ax[0].set_ylabel("Y [AU]")
    ax[0].set_xlim(-1.25*np.max(abs(Rs1)),1.25*np.max(abs(Rs1)))
    ax[0].set_ylim(-1.25*np.max(abs(Rs1)),1.25*np.max(abs(Rs1)))
    ax[0].set_title(r"$\Delta t = 0.01 [yrs]$")

    ax[1].scatter(xs2[-1],ys2[-1],c='k',marker='*',s=100)
    ax[1].plot(xp2[ind2],yp2[ind2],c='k',alpha=0.3)
    ax[1].scatter(xp2[np.max(ind2)],yp2[np.max(ind2)],c='k',s=100)
    ax[1].set_xlabel("X [AU]")
    ax[1].set_ylabel("Y [AU]")
    ax[1].set_xlim(-1.25*np.max(abs(Rs1)),1.25*np.max(abs(Rs1)))
    ax[1].set_ylim(-1.25*np.max(abs(Rs1)),1.25*np.max(abs(Rs1)))
    ax[1].set_title(r"$\Delta t = 0.05 [yrs]$")

    ax[2].plot(times1[ind1],Es1[ind1],c='r',alpha=0.3)
    ax[2].plot(times2[ind2],Es2[ind2],c='k',alpha=0.3)
    ax[2].set_ylabel("Energy(t)")
    ax[2].set_xlabel("Time [yrs]")
    ax[2].set_xlim(0,np.max(np.concatenate([times1,times2])))
    Es = np.concatenate([Es1,Es2])
    ax[2].set_ylim(np.min(Es),np.max(Es))
    ax[2].set_title(r"$Energy(t)$")
    return fig

In [11]:
parameter_grid = list(range(1,1000)) #plot every 5th timestep
savefigures(makeplot,parameter_grid,outdir='./comet/')
render('./comet/','comet','gif',cleanup_type='rm')

saving figures


100%|█████████████████████████████████████████| 999/999 [12:23<00:00,  1.34it/s]
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora 

Framerate is None fps
Total Runtime is None s


Input #0, image2, from '%d.png':
  Duration: 00:00:39.96, start: 0.000000, bitrate: N/A
  Stream #0:0: Video: png, rgba(pc), 7283x2625 [SAR 11811:11811 DAR 7283:2625], 25 fps, 25 tbr, 25 tbn, 25 tbc
Stream mapping:
  Stream #0:0 -> #0:0 (png (native) -> png (native))
Press [q] to stop, [?] for help
[Parsed_palettegen_0 @ 0x5585c8126f80] 255(+1) colors generated out of 2855 colors; ratio=0.089317
[Parsed_palettegen_0 @ 0x5585c8126f80] Dupped color: FF0C0C0C
[Parsed_palettegen_0 @ 0x5585c8126f80] Dupped color: FFCDA0A0
[Parsed_palettegen_0 @ 0x5585c8126f80] Dupped color: FFCDA7A7
[Parsed_palettegen_0 @ 0x5585c8126f80] Dupped color: FFFFCCCC
[Parsed_palettegen_0 @ 0x5585c8126f80] Dupped color: FFFFDFDF
[Parsed_palettegen_0 @ 0x5585c8126f80] Dupped color: FFFFE6E6
Output #0, image2, to '../temp_palette_comet.png':
  Metadata:
    encoder         : Lavf58.76.100
  Stream #0:0: Video: png, rgba(pc, gbr/unknown/unknown, progressive), 16x16 [SAR 1:1 DAR 1:1], q=2-31, 200 kb/s, 25 fps, 25 tbn
 